In [3]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from keras.applications.vgg19 import VGG19
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from glob import glob

## File Path

In [4]:
train_path = 'Fruit/Training'
test_path = 'Fruit/Test'

## Image Dimension v Number Of Classes

In [5]:
img=load_img(train_path + "/Apple Golden 1/1_100.jpg")
x=img_to_array(img)
print(x.shape)

(100, 100, 3)


In [6]:
className = glob(train_path +'/*')
numberOfClass=len(className)
print("Number Of Class: ", numberOfClass)

Number Of Class:  43


## Data Augmentation

In [7]:
batch_size = 128

train_datagen=ImageDataGenerator(rescale=1./255,
                   shear_range=0.3,
                   horizontal_flip=True,
                   zoom_range=0.3)

test_datagen=ImageDataGenerator(rescale=1./255)     

train_generator = train_datagen.flow_from_directory(train_path,
                                                    target_size=(224,224),
                                                    batch_size=batch_size,
                                                    color_mode="rgb",
                                                    class_mode="categorical")

test_generator = test_datagen.flow_from_directory(test_path,
                                                    target_size=(224,224),
                                                    batch_size=batch_size,
                                                    color_mode="rgb",
                                                    class_mode="categorical")

Found 21387 images belonging to 43 classes.
Found 7156 images belonging to 43 classes.


## Model

In [8]:
vgg19 = VGG19(include_top=False, weights="imagenet", input_shape = (224, 224, 3))

model = Sequential(vgg19.layers)

model.add(Flatten())

model.add(Dense(128, activation = 'relu'))

model.add(Dropout(0.5))

model.add(Dense(43, activation = 'softmax'))

## Early Stopping v Model Checkpoint

In [10]:
model_checkpoint = ModelCheckpoint(
    filepath='transfer_learning_fruit.h5',
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

early = EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=0,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=False,
)

## Model Compile

In [9]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

## Model Fit

In [ ]:
epochs = 2

hist=model.fit_generator(generator = train_generator,
                         epochs = epochs,
                         validation_data = test_generator,
                         callbacks=[early, model_checkpoint])

## Data Visualization

In [ ]:

plt.plot(hist.history["loss"], label="Train Loss")
plt.plot(hist.history["val_loss"], label="Validation Loss")
plt.legend()
plt.show()

plt.figure()

plt.plot(hist.history["loss"], label="Train Accuracy")
plt.plot(hist.history["val_accuracy"], label="Validation Accuracy")
plt.legend()
plt.show()